In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from concurrent.futures import ThreadPoolExecutor
from geopy.geocoders import Photon
import folium
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
dataframe_aluguel = pd.read_parquet('dados_webscraping/dataframe_imoveis_aluguel_pronto_para_tratamento.parquet')
display(dataframe_aluguel.head(2))
dataframe_aluguel_tratado = dataframe_aluguel.drop_duplicates(subset='url', keep='first')
dataframe_aluguel_tratado = dataframe_aluguel_tratado[dataframe_aluguel_tratado["finalidade"] == "residencial"]


print("Quantidade de imóveis: ", dataframe_aluguel_tratado.shape[0])
print("Porcentagem de informações faltantes: ")
display(round(dataframe_aluguel_tratado.isna().sum()/dataframe_aluguel_tratado.shape[0] * 100, 2).astype(str) + " %")

,url,endereco,preco,area,andar,quartos,suites,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,condominio,elevador,jardim,quadra_esportiva,academia,finalidade,tipo
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,NaN,3.0,1.0,2,1982.0,470.0,False,False,False,False,True,False,False,residencial,apartamento
1,https://www.zapimoveis.com.br/imovel/aluguel-c...,"Rua Alice - Laranjeiras, Rio de Janeiro - RJ",20000.0,300.0,NaN,8.0,2.0,4,NaN,6558.0,False,False,False,False,False,False,False,residencial,casa


Quantidade de imóveis:  2539
Porcentagem de informações faltantes: 


url                   0.0 %
endereco             1.26 %
preco                1.54 %
area                 1.26 %
andar               61.52 %
quartos              5.32 %
suites              42.34 %
vagas_de_carro        0.0 %
valor_condominio     21.7 %
iptu                25.36 %
mobiliado             0.0 %
piscina               0.0 %
condominio            0.0 %
elevador              0.0 %
jardim                0.0 %
quadra_esportiva      0.0 %
academia              0.0 %
finalidade            0.0 %
tipo                  0.0 %
dtype: object

In [8]:
dataframe_aluguel_tratado = dataframe_aluguel_tratado.drop(columns=['andar', "suites"])
dataframe_aluguel_tratado = dataframe_aluguel_tratado.dropna()

print("Quantidade de imóveis: ", dataframe_aluguel_tratado.shape[0])

Quantidade de imóveis:  1608


In [9]:
# def _geocod_propertys_dataframe(dataframe, user_agent):

#     def _geocode_address(address, geolocator):
#         try:
#             geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)
#             location = geocode(address)
#             return location.latitude if location else None, location.longitude if location else None
#         except:
#             return None, None

#     geolocator = Nominatim(user_agent=user_agent, timeout=10)
#     addresses = dataframe['endereco'].tolist()

#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda addr: _geocode_address(addr, geolocator), addresses), total=len(addresses)))

#     dataframe['latitude'], dataframe['longitude'] = zip(*results)
#     return dataframe

# dataframe_aluguel_tratado = _geocod_propertys_dataframe(dataframe_aluguel_tratado, "Geopy Library")

# display(dataframe_aluguel_tratado.head(2))

dataframe_aluguel_ref = pd.read_parquet('dados_webscraping/dataframe_imoveis_aluguel_tratados_com_localizacao_bruto.parquet')[["url", "localizacao","latitude", "longitude"]].drop_duplicates(subset="url", keep="first")
dataframe_aluguel_tratado = dataframe_aluguel_tratado.merge(dataframe_aluguel_ref, on="url", how="left")
dataframe_final = dataframe_aluguel_tratado.dropna()

print(dataframe_final.shape)
display(dataframe_aluguel_tratado.head(2))

(1448, 20)


,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,condominio,elevador,jardim,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,False,False,True,False,False,residencial,apartamento,"Avenida Epitácio Pessoa, Lagoa, Rio de Janeiro...",-22.978197,-43.199374
1,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Salvador Allende, 931 - Barra da Tijuc...",2300.0,50.0,2.0,1,700.0,75.0,False,False,False,False,False,False,False,residencial,apartamento,"Avenida Salvador Allende, Barra da Tijuca, Rio...",-22.989682,-43.417630


In [10]:
geometry = [Point(xy) for xy in zip(dataframe_final['longitude'], dataframe_final['latitude'])]
dataframe_final = gpd.GeoDataFrame(dataframe_final, geometry=geometry)
dataframe_final = dataframe_final.set_crs("EPSG:4326") 

print(dataframe_final.shape)
dataframe_final.head(2)

(1448, 21)


,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,elevador,jardim,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude,geometry
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,...,False,True,False,False,residencial,apartamento,"Avenida Epitácio Pessoa, Lagoa, Rio de Janeiro...",-22.978197,-43.199374,POINT (-43.19937 -22.9782)
1,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Salvador Allende, 931 - Barra da Tijuc...",2300.0,50.0,2.0,1,700.0,75.0,False,False,...,False,False,False,False,residencial,apartamento,"Avenida Salvador Allende, Barra da Tijuca, Rio...",-22.989682,-43.417630,POINT (-43.41763 -22.98968)


In [11]:
#Associando os imóveis aos bairros

limites_bairros = gpd.read_file('dados_demograficos/Limite_de_Bairros/Limite_de_Bairros.shp')
limites_bairros = limites_bairros.to_crs(epsg=4326)

dataframe_final = gpd.sjoin(dataframe_final, limites_bairros[["nome", "geometry"]], how="left", predicate="within").drop(columns=['index_right'])

In [12]:
def get_properties_near_especific_points(dataframe, dataframe_ref, nome_coluna_referencia):

    dataframe = dataframe.to_crs(epsg=32633)
    dataframe_ref = dataframe_ref.to_crs(epsg=32633)

    dataframe[nome_coluna_referencia] = dataframe["geometry"].apply(
        lambda ponto: dataframe_ref["geometry"].distance(ponto).min())
    dataframe = dataframe.to_crs(epsg=4326)
    
    return dataframe

paradas_de_metro = gpd.read_file('dados_demograficos/Estações_Metrô/Estações_Metrô.shp')

dataframe_final = get_properties_near_especific_points(dataframe_final, paradas_de_metro, "distancia_metro")
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,...,False,False,residencial,apartamento,"Avenida Epitácio Pessoa, Lagoa, Rio de Janeiro...",-22.978197,-43.199374,POINT (-43.19937 -22.9782),Lagoa,948.063010
1,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Salvador Allende, 931 - Barra da Tijuc...",2300.0,50.0,2.0,1,700.0,75.0,False,False,...,False,False,residencial,apartamento,"Avenida Salvador Allende, Barra da Tijuca, Rio...",-22.989682,-43.417630,POINT (-43.41763 -22.98968),Barra da Tijuca,17900.901142


In [13]:
escolas_municipais = gpd.read_file('dados_demograficos/Escolas_Municipais/Escolas_Municipais.shp')

dataframe_final = get_properties_near_especific_points(dataframe_final, escolas_municipais, "distancia_escola")
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,academia,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro,distancia_escola
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,...,False,residencial,apartamento,"Avenida Epitácio Pessoa, Lagoa, Rio de Janeiro...",-22.978197,-43.199374,POINT (-43.19937 -22.9782),Lagoa,948.063010,437.351027
1,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Salvador Allende, 931 - Barra da Tijuc...",2300.0,50.0,2.0,1,700.0,75.0,False,False,...,False,residencial,apartamento,"Avenida Salvador Allende, Barra da Tijuca, Rio...",-22.989682,-43.417630,POINT (-43.41763 -22.98968),Barra da Tijuca,17900.901142,2474.507407


In [14]:
unidade_saude = gpd.read_file('dados_demograficos/Unidades_de_Saúde_Municipais/Unidades_de_Saúde_Municipais.shp')

dataframe_final = get_properties_near_especific_points(dataframe_final, unidade_saude, "distancia_unidade_saude")
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro,distancia_escola,distancia_unidade_saude
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,...,residencial,apartamento,"Avenida Epitácio Pessoa, Lagoa, Rio de Janeiro...",-22.978197,-43.199374,POINT (-43.19937 -22.9782),Lagoa,948.063010,437.351027,767.352721
1,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Salvador Allende, 931 - Barra da Tijuc...",2300.0,50.0,2.0,1,700.0,75.0,False,False,...,residencial,apartamento,"Avenida Salvador Allende, Barra da Tijuca, Rio...",-22.989682,-43.417630,POINT (-43.41763 -22.98968),Barra da Tijuca,17900.901142,2474.507407,2499.083937


In [15]:
indice_de_desenvolvimento = gpd.read_file('dados_demograficos/Índice_de_Desenvolvimento/Índice_de_Desenvolvimento.shp')
indice_de_desenvolvimento = indice_de_desenvolvimento.to_crs(epsg=4326)

dataframe_final = gpd.sjoin(dataframe_final, indice_de_desenvolvimento[["indic_rend", "indic_lixo", "indic_esgo", "indic_agua","geometry"]], how="left", predicate="within").drop(columns=['index_right'])
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,longitude,geometry,nome,distancia_metro,distancia_escola,distancia_unidade_saude,indic_rend,indic_lixo,indic_esgo,indic_agua
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,...,-43.199374,POINT (-43.19937 -22.9782),Lagoa,948.063010,437.351027,767.352721,20.550754,99.987981,99.915865,99.987981
1,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Salvador Allende, 931 - Barra da Tijuc...",2300.0,50.0,2.0,1,700.0,75.0,False,False,...,-43.417630,POINT (-43.41763 -22.98968),Barra da Tijuca,17900.901142,2474.507407,2499.083937,16.847244,99.972639,88.609846,99.183067


In [16]:
dataframe_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1448 entries, 0 to 1607
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   url                      1448 non-null   object  
 1   endereco                 1448 non-null   object  
 2   preco                    1448 non-null   float64 
 3   area                     1448 non-null   float64 
 4   quartos                  1448 non-null   float64 
 5   vagas_de_carro           1448 non-null   int64   
 6   valor_condominio         1448 non-null   float64 
 7   iptu                     1448 non-null   float64 
 8   mobiliado                1448 non-null   bool    
 9   piscina                  1448 non-null   bool    
 10  condominio               1448 non-null   bool    
 11  elevador                 1448 non-null   bool    
 12  jardim                   1448 non-null   bool    
 13  quadra_esportiva         1448 non-null   bool    
 14  acade

In [17]:
dataframe_final.to_parquet('dados_webscraping/dataframe_imoveis_aluguel_pronto_para_analise.parquet')